## Marhub data cleaning
Merged data from mona and irap

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
from scipy.stats import pearsonr
from sklearn.linear_model import LogisticRegression 
import statsmodels.api as sm


#### Reading data

In [2]:
path = "../Final Soft Launch Results/Current Source Data/"

In [3]:
d1="Data from IRAP 01072020.xlsx"
d2="Merge for Vulnerability Screening.xlsx"
d3="Raw Data from Mona 01172020.xlsx"

In [4]:
irap = pd.read_excel (path+d1)
mona = pd.read_excel (path+d3)

In [5]:
irap.head()

,Mona Unique Session ID,Mona Unique User ID,Created on,Created by,Pipeline,Date of Referral,First Referral,In IRAP Process,Screening Stage,Screener,...,Legal Assistance Identified,ILI Assistance Provided,Mona Case Type Identification,Mona Follow up Needed,Vulnerability Threshold,Screening Decision,Date of Screening Decision,Flags,Client Location (Country),Client Nationality
0,ce81140c-7083-4463-90ed-ad36d1a41011.2019-07-2...,ce81140c-7083-4463-90ed-ad36d1a41011,2019-12-11 06:02:17,Brooke Sauro,Mona,2019-07-25,No,No,Screening Complete,Michaela Gallien,...,NaN,NaN,Accurate,No,Below,Cannot Represent,2019-08-12,NaN,NaN,NaN
1,92952aa434524f549543d3fcc79054fe20190804T02434...,92952aa4-3452-4f54-9543-d3fcc79054fe,2019-12-11 06:02:17,Brooke Sauro,Mona,2019-08-03,No,Yes,Screening Complete,Manal ElKhoury,...,Vulnerability RST Referral,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN
2,8a4a87a6eeeb43019d820644a8a7534620190802T07350...,8a4a87a6-eeeb-4301-9d82-0644a8a75346,2019-12-11 06:02:17,Brooke Sauro,Mona,2019-08-02,No,No,Screening,Tiba Fatli,...,Vulnerability RST Referral,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN
3,758240c9578a4e29b66bd1d95de9cdc620190802T21293...,758240c9-578a-4e29-b66b-d1d95de9cdc6,2019-12-11 06:02:17,Brooke Sauro,Mona,2019-08-02,No,No,Screening Complete,Tania El Khoury,...,Vulnerability RST Referral,NaN,Accurate,No,Below,Cannot Represent,2019-12-20,NaN,NaN,NaN
4,55865e62-97b3-4656-b470-7eaaa6a517cf.2019-07-2...,55865e62-97b3-4656-b470-7eaaa6a517cf,2019-12-11 06:02:16,Brooke Sauro,Mona,2019-07-26,No,No,Screening Complete,Tania El Khoury,...,Vulnerability RST Referral,Non-Legal Services Information,Accurate,Yes,Below,Cannot Represent,2019-12-17,NaN,NaN,NaN


#### generate variables and merge datasets

In [5]:
irap['process_days'] = irap['Date of Screening Decision']-irap['Date of Screening Start']
irap['process_days'] = irap['process_days'].astype('timedelta64[D]')
# irap['process_daysA'] = irap['Date of Screening Start']-irap['Date of Referral']
# irap['process_daysA'] = irap['process_daysA'].astype('timedelta64[D]')
# irap['process_daysC'] = irap['Date of Screening Decision']-irap['Date of Referral']
# irap['process_daysC'] = irap['process_daysC'].astype('timedelta64[D]')

In [6]:
mona = mona.rename({'Contact UUID': 'Mona Unique User ID'}, axis=1)

In [7]:
full = pd.merge(irap, mona, on='Mona Unique User ID',how="outer").dropna(axis=1, how='all')

In [8]:
full.shape

(7676, 337)

In [9]:
# full['Field:Gender'].replace(('Male', 'Female'), (1, 0), inplace=True)
# full['Vulnerability Threshold'].replace(('Above', 'Below'), (1, 0), inplace=True)

In [10]:
def one_hot(dataframe,column_name):
    gen_onehot_features = pd.get_dummies(dataframe[column_name])
    result = pd.concat([dataframe, gen_onehot_features],axis=1)
    return result

In [11]:
# full['Field:Nationality1'] = 'nationality_' + full['Field:Nationality1']
# full = one_hot(full,'Field:Nationality1')


In [12]:
# full['Field:Current Country'] = 'curr_country_' + full['Field:Current Country']
# full = one_hot(full,'Field:Current Country')

In [13]:
full.shape

(7676, 337)

In [14]:
# numeric = full.select_dtypes(exclude=['object','datetime64[ns]'])

In [15]:
full.head()

,Mona Unique Session ID,Mona Unique User ID,Created on,Created by,Pipeline,Date of Referral,First Referral,In IRAP Process,Screening Stage,Screener,...,Field:Aggr 3 Threats Since Moving,Field:Aggr 2 Reason Unable to Move,Field:Aggr 3 Reason Unable to Move,Field:Open Ended Other Threats,Field:RFR Submission,Field:Vulnerability Assessed,Field:Handoff Timestamp,Field:Prescreening Complete,Field:Pause Handoffs,Field:Lebanon Location
0,ce81140c-7083-4463-90ed-ad36d1a41011.2019-07-2...,ce81140c-7083-4463-90ed-ad36d1a41011,2019-12-11 06:02:17,Brooke Sauro,Mona,2019-07-25,No,No,Screening Complete,Michaela Gallien,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Yes,NaN,NaN
1,92952aa434524f549543d3fcc79054fe20190804T02434...,92952aa4-3452-4f54-9543-d3fcc79054fe,2019-12-11 06:02:17,Brooke Sauro,Mona,2019-08-03,No,Yes,Screening Complete,Manal ElKhoury,...,NaN,NaN,NaN,NaN,NaN,Vulnerability Assessed,2019-08-08T00:08:27.754958+03:00,Yes,Yes,NaN
2,92952aa434524f549543d3fcc79054fe20190806T11414...,92952aa4-3452-4f54-9543-d3fcc79054fe,2019-10-07 08:21:12,Brooke Sauro,Mona,2019-08-08,Yes,No,Screening,Manal ElKhoury,...,NaN,NaN,NaN,NaN,NaN,Vulnerability Assessed,2019-08-08T00:08:27.754958+03:00,Yes,Yes,NaN
3,8a4a87a6eeeb43019d820644a8a7534620190802T07350...,8a4a87a6-eeeb-4301-9d82-0644a8a75346,2019-12-11 06:02:17,Brooke Sauro,Mona,2019-08-02,No,No,Screening,Tiba Fatli,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,758240c9578a4e29b66bd1d95de9cdc620190802T21293...,758240c9-578a-4e29-b66b-d1d95de9cdc6,2019-12-11 06:02:17,Brooke Sauro,Mona,2019-08-02,No,No,Screening Complete,Tania El Khoury,...,NaN,NaN,NaN,NaN,NaN,Vulnerability Assessed,2019-08-02T22:03:58.358208+03:00,NaN,NaN,NaN


In [16]:
def label_process_day_1(row):
    if row['process_days'] >= 1 :
        return 1
    if row['process_days'] == 0 :
        return 0
    else: return np.nan
def label_process_day_7(row):
    if row['process_days'] >= 7 :
        return 1
    if row['process_days'] <7 :
        return 0
    else: return np.nan
# def label_process_dayA_1(row):
#     if row['process_daysA'] >= 1 :
#         return 1
#     if row['process_daysA'] == 0 :
#         return 0
#     else: return np.nan
# def label_process_dayC_1(row):
#     if row['process_daysC'] >= 1 :
#         return 1
#     if row['process_daysC'] == 0 :
#         return 0
#     else: return np.nan

In [17]:
full['process_day_>=1'] = full.apply (lambda row: label_process_day_1(row), axis=1)
full['process_day_>=7'] = full.apply (lambda row: label_process_day_7(row), axis=1)
# full['process_dayA_>=1'] = full.apply (lambda row: label_process_dayA_1(row), axis=1)
# full['process_dayC_>=1'] = full.apply (lambda row: label_process_dayC_1(row), axis=1)
full['Medical_X_LPPN'] = full['Field:Medical Needs Score'] * full['Field:LPPN Score']

###  Check for Null values


field names, values, and percentage missing

In [18]:
full.shape

(7676, 340)

In [19]:
null_count = pd.Series.to_frame(full.isna().sum())
# null_count.rename(columns={"0": "null_count"})
null_count.columns = ["null_counts"]
null_count["total_counts"] = full.shape[0]
null_count["null_percent"] = np.round(null_count["null_counts"] / full.shape[0] , 5)
null_count["dtype"] = pd.Series.to_frame(full.dtypes)
null_count["sample_value"] = full.mode()[:1].T
null_count

,null_counts,total_counts,null_percent,dtype,sample_value
Mona Unique Session ID,6590,7676,0.85852,object,003b4c8a-fa26-4d7c-bb47-c27109b757e3.2019-07-2...
Mona Unique User ID,199,7676,0.02592,object,217a088b-d4ce-408f-a5d8-c4eedc4aeb9f
Created on,6590,7676,0.85852,datetime64[ns],2019-12-11 06:01:52
Created by,6590,7676,0.85852,object,Brooke Sauro
Pipeline,6590,7676,0.85852,object,Mona
...,...,...,...,...,...
Field:Pause Handoffs,4811,7676,0.62676,object,Yes
Field:Lebanon Location,7019,7676,0.91441,object,نعم.
process_day_>=1,6898,7676,0.89865,float64,0
process_day_>=7,6890,7676,0.89760,float64,0


In [20]:
null_count.sort_values(['null_counts'],ascending = False)

,null_counts,total_counts,null_percent,dtype,sample_value
Field:RFR within deadline,7675,7676,0.99987,object,jsv80uRFRv0wnixf
Field:I130 Info Response,7675,7676,0.99987,object,كيف ابدأ بتفعيل الطلب المقدم من جهة اختي
Field:Mexico Passport Type,7675,7676,0.99987,object,Replacement
Field:DAP as Relative of SIV,7675,7676,0.99987,object,DAP as Relative of SIV
Field:Aggr 3 Reason Unable to Move,7674,7676,0.99974,object,Reason Unable to Move: عدم تنقلي سببه انه بعد ...
...,...,...,...,...,...
Language,1055,7676,0.13744,object,ara
URN:Facebook,887,7676,0.11555,float64,1.22442e+15
Name,296,7676,0.03856,object,محمد
Mona Unique User ID,199,7676,0.02592,object,217a088b-d4ce-408f-a5d8-c4eedc4aeb9f


In [21]:
# null_count.to_csv("null_counts_full.csv")

### Data cleanup

In [22]:
full_clean = full.copy()

In [23]:
pd.set_option("max_rows", None)

pd.reset_option("max_rows")


In [24]:
full['Field:Gender']

0         Male
1         Male
2         Male
3       Female
4         Male
         ...  
7671       NaN
7672       NaN
7673       NaN
7674       NaN
7675       NaN
Name: Field:Gender, Length: 7676, dtype: object

#### change to / create new binary field indicating (null vs. not null)

In [25]:
def create_binary_field(df, column_name):
    df[column_name+"_binary"] = df[column_name].apply(lambda x: 0 if pd.isnull(x) else 1)

In [26]:
def changeto_binary_field(df, column_name):
    if df[column_name].isna().sum() == 0:
        print("[log]: no change")
        return
    
    df[column_name] = df[column_name].apply(lambda x: 0 if pd.isnull(x) else 1)
    print("[log]: Updated to binary null/not null")

In [27]:
'Field:I130 Info Rating'[6:]

'I130 Info Rating'

In [28]:
for i in ['Field:Aggr 3 Reason Unable to Move',
'Field:I130 Info Rating',
'Field:Aggr 3 Mst Svr Threat',
'Field:DAPI130 Info Rating',
'Field:Aggr 2 Mst Svr Threat',
'Field:Aggr 3 Mst Svr Only Threats',
'Field:P3 Info Rating',
'Field:Aggr 3 Mst Svr Attack',
'Field:Aggr 3 Attack Within Last 6 Months',
'Field:Aggr 2 Reason Unable to Move',
'Field:Aggr 3 Threat Within Last 6 Months',
'Field:Women and Girls Severity Assessment',
'Field:DAPEV Info Rating',
'Field:Canadian Sponsorship Info Rating',
'Field:Aggr 1 Reason Unable to Move',
'Field:Nationality2',
'Field:Email Address1',
'Field:I 130 F',
'Field:Worked for USG',
'Field:Aggr 1 Mst Svr Only Threats',
         'Field:Age',
'Field:End Feedback Helpfulness Rating',
'Field:End Feedback Recommend Rating',
'Field:German Reunification Info Rating',
'Field:Lebanon UNHCR Info Rating',
'Field:Medical Needs Score',
'Field:Years in Current Country',
'Field:Traumatic Incident 1 Assessment',
'Field:Aggressor 2 Score',
'Field:Medical Condition 1 Assessment',
'Field:Medical Condition 1 Score',
'Field:Medical Condition 2 Score',
'Field:Medical Condition 3 Score',
'Field:Aggr 1 SVT Rating',
'Field:Aggressor 1 Score',
'Field:Medical Condition 2 Assessment',
'Field:Aggr 1 Threat Within Last 6 Months',
'Field:ICORN Info Rating',
'Field:Aggr 1 Attack Within Last 6 Months',
'Field:Traumatic Incident 2 Assessment',
'Field:Aggr 1 Mst Svr Attack',
'Field:USRAP Info Rating',
'Field:Aggr 2 SVT Rating',
'Field:Medical Condition 3 Assessment',
'Field:Aggr 2 Threat Within Last 6 Months',
'Field:Aggr 2 Attack Within Last 6 Months',
'Field:Aggr 2 Mst Svr Only Threats',
'Field:Aggr 1 Mst Svr Threat',
'Field:Aggr 3 SVT Rating',
'Field:Aggressor 3 Score',
'Field:Aggr 2 Mst Svr Attack']:
    changeto_binary_field(full_clean,i)
    print(full[i].value_counts())
    print(full_clean[i].value_counts())
    print("-"*30)

[log]: Updated to binary null/not null
Reason Unable to Move: عدم تنقلي سببه انه بعد المراجعات الى الافرع الامنيه اخبروني باني اذا غيرت سكني يجب ابلاغهم على الفور    1
Reason Unable to Move: ولك خلص                                                                                                 1
Name: Field:Aggr 3 Reason Unable to Move, dtype: int64
0    7674
1       2
Name: Field:Aggr 3 Reason Unable to Move, dtype: int64
------------------------------
[log]: Updated to binary null/not null
4.0    1
5.0    1
Name: Field:I130 Info Rating, dtype: int64
0    7674
1       2
Name: Field:I130 Info Rating, dtype: int64
------------------------------
[log]: Updated to binary null/not null
5                                     1
Severity of Most Serious Threat: 5    1
Severity of Most Serious Threat: 4    1
Name: Field:Aggr 3 Mst Svr Threat, dtype: int64
0    7673
1       3
Name: Field:Aggr 3 Mst Svr Threat, dtype: int64
------------------------------
[log]: Updated to binary null/not null
5.0

[log]: Updated to binary null/not null
5.0    14
3.0     7
1.0     7
4.0     5
2.0     2
0.0     1
Name: Field:DAPEV Info Rating, dtype: int64
0    7640
1      36
Name: Field:DAPEV Info Rating, dtype: int64
------------------------------
[log]: Updated to binary null/not null
5                                                          289
3                                                          156
1                                                          125
4                                                           95
2                                                           94
١                                                            7
٢                                                            2
لم افهم                                                      2
٣                                                            2
مقبول الى حد ما                                              1
٥                                                            1
5 5                           

[log]: Updated to binary null/not null
Severity of Most Serious Attack: 5    34
5                                     32
1                                     25
3                                     22
Severity of Most Serious Attack: 3    20
Severity of Most Serious Attack: 4    17
Severity of Most Serious Attack: 1    14
4                                     14
2                                     14
Severity of Most Serious Attack: 2     9
٣                                      1
١                                      1
٥                                      1
Name: Field:Aggr 1 Mst Svr Attack, dtype: int64
0    7472
1     204
Name: Field:Aggr 1 Mst Svr Attack, dtype: int64
------------------------------
[log]: Updated to binary null/not null
5                                                                                               86
3                                                                                               31
1                                           

#### Specific cleaner for adult and marriage variables

In [29]:
def create_adult_field(df, column_name='Field:Minor or Adult'):
    if "Adult" not in df.columns:
        df["Adult"] = df[column_name].apply(lambda x: 1 if x=="Adult" else 0)
        df.drop(columns=[column_name])
        print("Done")
    else:
        print("Already created Adult column")

In [30]:
create_adult_field(full_clean)

Done


In [31]:
def create_married_field(df, column_name='Field:Marital Status'):
    if "Married" not in df.columns:
        df["Married"] = df[column_name].apply(lambda x: 1 if x=="Married" else 0)
        df.drop(columns=[column_name])
        print("Done")
    else:
        print("Already created Married column")

In [32]:
create_married_field(full_clean)

Done


#### String cleaning to numeric

In [34]:
import re
def string_to_int(string):
    end_num = re.compile('(\d)\s*.*$')
    try:
        return(int(end_num.findall(string)[0]))
    except: return(np.nan)

In [35]:
string_to_int("3.0")

3

In [36]:
def convert_to_numeric(df, column_name):
    new = df[column_name].apply(lambda x: string_to_int(str(x)))
    df[column_name] = new

In [37]:
for i in ['Field:USRAP Info Rating',
'Field:Aggr 1 Mst Svr Attack',
'Field:Traumatic Incident 2 Assessment',
'Field:Aggr 1 Attack Within Last 6 Months',
'Field:ICORN Info Rating',
'Field:Aggr 1 Threat Within Last 6 Months',
'Field:Medical Condition 2 Assessment',
'Field:Traumatic Incident 1 Assessment',
'Field:Lebanon UNHCR Info Rating',
'Field:German Reunification Info Rating',
'Field:Aggr 1 Mst Svr Threat',
'Field:Aggr 2 Mst Svr Only Threats',
'Field:Aggr 2 Attack Within Last 6 Months',
'Field:Aggr 2 Threat Within Last 6 Months',
'Field:Medical Condition 3 Assessment',
'Field:Aggr 2 SVT Rating',
'Field:Aggr 3 SVT Rating',
'Field:Aggressor 1 Score',
'Field:Aggr 1 SVT Rating',
'Field:Medical Condition 3 Score',
'Field:Medical Condition 2 Score',
'Field:Medical Condition 1 Assessment',
'Field:Medical Condition 1 Score',
'Field:Aggressor 2 Score',
'Field:Years in Current Country',
'Field:Medical Needs Score',
'Field:End Feedback Recommend Rating',
'Field:End Feedback Helpfulness Rating',
'Field:Age',
'Field:Aggr 2 Mst Svr Attack',
'Field:Aggressor 3 Score']:
    convert_to_numeric(full_clean,i)
    print(full[i].value_counts())
    print(full_clean[i].value_counts())
    print("-"*30)

5                                                                                               86
3                                                                                               31
1                                                                                               31
4                                                                                               26
2                                                                                               21
بس  انا ما دقولي  كرمال السفر انا قدمت وما ردولي خبر ... اذا فيكن تساعدوني وتقدمولي  طلب سفر     1
١                                                                                                1
لايوجد جديد                                                                                      1
كلا                                                                                              1
Name: Field:USRAP Info Rating, dtype: int64
0    7477
1     199
Name: Field:USRAP Info Rating, dtype: int64
-

Name: Field:Aggr 2 Threat Within Last 6 Months, dtype: int64
0    7578
1      98
Name: Field:Aggr 2 Threat Within Last 6 Months, dtype: int64
------------------------------
5.0    46
3.0    39
4.0    38
1.0    15
2.0     9
Name: Field:Medical Condition 3 Assessment, dtype: int64
0    7529
1     147
Name: Field:Medical Condition 3 Assessment, dtype: int64
------------------------------
Aggressor SVT Rating: 0    48
0                          39
5                          23
Aggressor SVT Rating: 5    22
Aggressor SVT Rating: 3    10
3                           9
Aggressor SVT Rating: 4     6
Aggressor SVT Rating: 1     5
4                           3
Aggressor SVT Rating: 2     3
2                           3
1                           1
Name: Field:Aggr 2 SVT Rating, dtype: int64
0    7504
1     172
Name: Field:Aggr 2 SVT Rating, dtype: int64
------------------------------
Aggressor SVT Rating: 0    7
5                          6
0                          4
Aggressor SVT Rating: 2   

#### Arabic to English

In [38]:
def convert_arabic_boolean(df, column_name, as_yes, as_no):
    if as_yes not in df[column_name].values and as_no not in df[column_name].values:
        print("[log]: no change")
        return
#     new = df[column_name].apply(lambda x: "Yes" if x==as_yes else ("No" if x==as_no else (np.nan if x!="Yes" and x!="No" else x)))
    new = df[column_name].apply(lambda x: "Yes" if x==as_yes else "No")

    df[column_name] = new
    print("[log]: converted "+column_name)

In [39]:
arrabic_no = "لا"

In [40]:
arrabic_yes = "نعم"

In [41]:
for i in ['Field:Medical Condition 3 Inc Treatment',
'Field:Medical Condition 3 SVT',
'Field:Medical Condition 3 Treatment Abroad',
'Field:Traumatic Incident 2 Ong Mdl Needs',
'Field:Medical Condition 2 Inc Treatment',
'Field:Medical Condition 2 SVT',
'Field:Medical Condition 2 Treatment Abroad',
'Field:Medical Condition 1 Inc Treatment',
'Field:Medical Condition 1 SVT',
'Field:Medical Condition 1 Treatment Abroad',
'Field:Traumatic Incident 1 Ong Mdl Needs']:
    convert_arabic_boolean(full_clean,i,arrabic_yes,arrabic_no)
    print(full[i].value_counts())
    print(full_clean[i].value_counts())
    print("-"*30)

[log]: converted Field:Medical Condition 3 Inc Treatment
لا     99
نعم    46
No      2
Name: Field:Medical Condition 3 Inc Treatment, dtype: int64
No     7630
Yes      46
Name: Field:Medical Condition 3 Inc Treatment, dtype: int64
------------------------------
[log]: converted Field:Medical Condition 3 SVT
لا     100
نعم     45
Yes      2
Name: Field:Medical Condition 3 SVT, dtype: int64
No     7631
Yes      45
Name: Field:Medical Condition 3 SVT, dtype: int64
------------------------------
[log]: converted Field:Medical Condition 3 Treatment Abroad
نعم        106
لا أعلم     35
لا           4
Yes          2
Name: Field:Medical Condition 3 Treatment Abroad, dtype: int64
No     7570
Yes     106
Name: Field:Medical Condition 3 Treatment Abroad, dtype: int64
------------------------------
[log]: converted Field:Traumatic Incident 2 Ong Mdl Needs
لا     230
نعم     99
No       1
Name: Field:Traumatic Incident 2 Ong Mdl Needs, dtype: int64
No     7577
Yes      99
Name: Field:Traumatic Inci

#### Custom convert null vs. not null

In [42]:
# convert null
def convert_to(df, column_name, not_null_as, null_as):
    if df[column_name].isna().sum() == 0:
        print("[log]: no change")
        return
    new = df[column_name].apply(lambda x: not_null_as if not pd.isnull(x) else null_as)
    df[column_name] = new
    print("[log]: converted "+column_name)

In [43]:
for i in ['Field:I 130 IR',
'Field:SIV Follow to Join',
'Field:P 3',
'Field:SIV',
'Field:DAP via EV',
'Field:DAP as Relative of SIV',
'Field:EO Waiver',
'Field:SIV 101a',
'Field:SIV 1059',
'Field:DAP via I 130',
'Field:ICORN CPJ',
'Field:Canadian Private Sponsorship',
'Field:Germany Family Reunification']:
    convert_to(full_clean,i,"Eligible","Not Eligible")
    print(full[i].value_counts())
    print(full_clean[i].value_counts())
    print("-"*30)

[log]: converted Field:I 130 IR
I-130 IR    3
Name: Field:I 130 IR, dtype: int64
Not Eligible    7673
Eligible           3
Name: Field:I 130 IR, dtype: int64
------------------------------
[log]: converted Field:SIV Follow to Join
SIV Follow to Join    3
Name: Field:SIV Follow to Join, dtype: int64
Not Eligible    7673
Eligible           3
Name: Field:SIV Follow to Join, dtype: int64
------------------------------
[log]: converted Field:P 3
P-3    8
Name: Field:P 3, dtype: int64
Not Eligible    7668
Eligible           8
Name: Field:P 3, dtype: int64
------------------------------
[log]: converted Field:SIV
SIV    12
Name: Field:SIV, dtype: int64
Not Eligible    7664
Eligible          12
Name: Field:SIV, dtype: int64
------------------------------
[log]: converted Field:DAP via EV
DAP via EV    24
Name: Field:DAP via EV, dtype: int64
Not Eligible    7652
Eligible          24
Name: Field:DAP via EV, dtype: int64
------------------------------
[log]: converted Field:DAP as Relative of SIV

In [44]:
for i in ['Field:Aggr 3 Threats Since Moving',
'Field:RFR Submission',
'Field:US Prior Admission',
'Field:Aggr 3 Moved to Avoid Aggr',
'Field:Testing Only',
'Field:Family Worked for USG',
'ILI Assistance Provided ',
'Field:RFR',
'Field:Aggr 2 Moved to Avoid Aggr',
'Flags',
'Field:Aggr 1 Threats Since Moving',
'Field:USRAP Access',
'Field:Aggr 1 Moved to Avoid Aggr',
'Field:Multiple Incidents',
'Field:Vulnerability Assessed',
'URN:Telegram',
'Field:Lack of Durable Solutions',
'Field:Women and Girls at Risk',
'Field:SVT',
'Field:LGBT',
'Field:Hired Lawyer',
'Field:Pause Handoffs',
'Field:Prescreening Complete',
'Field:Ongoing USRAP',
'Field:Previous US Resettlement',
'URN:Facebook']:
    convert_to(full_clean,i,"Yes","No")
    print(full[i].value_counts())
    print(full_clean[i].value_counts())
    print("-"*30)

[log]: converted Field:Aggr 3 Threats Since Moving
Received Threats Since Moving: Yes    11
Name: Field:Aggr 3 Threats Since Moving, dtype: int64
No     7665
Yes      11
Name: Field:Aggr 3 Threats Since Moving, dtype: int64
------------------------------
[log]: converted Field:RFR Submission
RFR Submission    11
Name: Field:RFR Submission, dtype: int64
No     7665
Yes      11
Name: Field:RFR Submission, dtype: int64
------------------------------
[log]: converted Field:US Prior Admission
RTD/RRV    15
Name: Field:US Prior Admission, dtype: int64
No     7661
Yes      15
Name: Field:US Prior Admission, dtype: int64
------------------------------
[log]: converted Field:Aggr 3 Moved to Avoid Aggr
Moved to Avoid This Aggressor: Yes       11
Moved to Avoid This Aggressor: Unable     9
Name: Field:Aggr 3 Moved to Avoid Aggr, dtype: int64
No     7656
Yes      20
Name: Field:Aggr 3 Moved to Avoid Aggr, dtype: int64
------------------------------
[log]: converted Field:Testing Only
Yes    21
Nam

In [45]:
convert_to(full_clean,"Vulnerability Threshold","Above","Below")

[log]: converted Vulnerability Threshold


In [46]:
convert_to(full_clean,'Field:RFR within deadline',"Yes","Not Applicable")

[log]: converted Field:RFR within deadline


#### Convert single variable formats

In [47]:
# convert 'Field:Aggr 2 Threats Since Moving'
def convert_threats(df, column_name='Field:Aggr 2 Threats Since Moving'):
    if df[column_name].isna().sum() == 0:
        print("[log]: no change")
        return
    new = df[column_name].apply(lambda x: "Yes" if x=="Received Threats Since Moving: Yes" else ("No" if x == "Received Threats Since Moving: No" else "Not Applicable"))
    df[column_name] = new
    print("[log]: converted "+column_name)

In [48]:
convert_threats(full_clean)
print(full['Field:Aggr 2 Threats Since Moving'].value_counts())
print(full_clean['Field:Aggr 2 Threats Since Moving'].value_counts())

[log]: converted Field:Aggr 2 Threats Since Moving
Received Threats Since Moving: Yes    19
Received Threats Since Moving: No     15
Name: Field:Aggr 2 Threats Since Moving, dtype: int64
Not Applicable    7642
Yes                 19
No                  15
Name: Field:Aggr 2 Threats Since Moving, dtype: int64


In [49]:
# convert 'Field:End Feedback Follow Up Conversation'
def convert_follow_up(df, column_name='Field:End Feedback Follow Up Conversation'):
    if df[column_name].isna().sum() == 0:
        print("[log]: no change")
        return
    new = df[column_name].apply(lambda x: x if not pd.isnull(x) else "Not Applicable")
    df[column_name] = new
    print("[log]: converted "+column_name)

In [50]:
convert_follow_up(full_clean)

[log]: converted Field:End Feedback Follow Up Conversation


In [51]:
full_clean = full_clean[full_clean['Field:Testing Only']=="No"]
full_clean = full_clean.drop(columns=['Field:Testing Only'])

### Cleaned data Output 

In [58]:
cleaned = full_clean[['Vulnerability Threshold',
'Field:Aggr 3 Threats Since Moving',
'Field:RFR Submission',
'Field:US Prior Admission',
'Field:Aggr 3 Moved to Avoid Aggr',
'Field:Family Worked for USG',
'ILI Assistance Provided ',
'Field:RFR',
'Field:Aggr 2 Moved to Avoid Aggr',
'Flags',
'Field:Aggr 1 Threats Since Moving',
'Field:USRAP Access',
'Field:Aggr 1 Moved to Avoid Aggr',
'Field:Multiple Incidents',
'Field:Vulnerability Assessed',
'URN:Telegram',
'Field:Lack of Durable Solutions',
'Field:Women and Girls at Risk',
'Field:SVT',
'Field:LGBT',
'Field:Hired Lawyer',
'Field:Pause Handoffs',
'Field:Prescreening Complete',
'Field:Ongoing USRAP',
'Field:Previous US Resettlement',
'URN:Facebook',
'Field:RFR within deadline',
'Field:Aggr 2 Threats Since Moving',
'Field:End Feedback Follow Up Conversation',
'Field:I 130 IR',
'Field:SIV Follow to Join',
'Field:P 3',
'Field:SIV',
'Field:DAP via EV',
'Field:DAP as Relative of SIV',
'Field:EO Waiver',
'Field:SIV 101a',
'Field:SIV 1059',
'Field:DAP via I 130',
'Field:ICORN CPJ',
'Field:Canadian Private Sponsorship',
'Field:Germany Family Reunification',
'Field:Medical Condition 3 Inc Treatment',
'Field:Medical Condition 3 SVT',
'Field:Medical Condition 3 Treatment Abroad',
'Field:Traumatic Incident 2 Ong Mdl Needs',
'Field:Medical Condition 2 Inc Treatment',
'Field:Medical Condition 2 SVT',
'Field:Medical Condition 2 Treatment Abroad',
'Field:Medical Condition 1 Inc Treatment',
'Field:Medical Condition 1 SVT',
'Field:Medical Condition 1 Treatment Abroad',
'Field:Traumatic Incident 1 Ong Mdl Needs',
'Field:Handoff Timestamp',
'Date of Screening Decision',
'Created On',
'Field:USRAP Info Rating',
'Field:Aggr 1 Mst Svr Attack',
'Field:Traumatic Incident 2 Assessment',
'Field:Aggr 1 Attack Within Last 6 Months',
'Field:ICORN Info Rating',
'Field:Aggr 1 Threat Within Last 6 Months',
'Field:Medical Condition 2 Assessment',
'Field:Traumatic Incident 1 Assessment',
'Field:Lebanon UNHCR Info Rating',
'Field:German Reunification Info Rating',
'Field:Aggr 1 Mst Svr Threat',
'Field:Aggr 2 Mst Svr Only Threats',
'Field:Aggr 2 Attack Within Last 6 Months',
'Field:Aggr 2 Threat Within Last 6 Months',
'Field:Medical Condition 3 Assessment',
'Field:Aggr 2 SVT Rating',
'Field:Aggr 3 SVT Rating',
'Field:Aggressor 1 Score',
'Field:Aggr 1 SVT Rating',
'Field:Years in Current Country',
'Field:Medical Needs Score',
'Field:End Feedback Recommend Rating',
'Field:End Feedback Helpfulness Rating',
'Field:Age',
'Field:Aggr 2 Mst Svr Attack',
'Field:Aggressor 3 Score',
'Adult',
'Married',
'Client Location (Country)',
'Client Nationality',
'Field:Current Country',
'Field:Nationality1',
'Field:Gender',
'Language',
'Field:Aggr 3 Reason Unable to Move',
'Field:I130 Info Rating',
'Field:Aggr 3 Mst Svr Threat',
'Field:DAPI130 Info Rating',
'Field:Aggr 2 Mst Svr Threat',
'Field:Aggr 3 Mst Svr Only Threats',
'Field:P3 Info Rating',
'Field:Aggr 3 Mst Svr Attack',
'Field:Aggr 3 Attack Within Last 6 Months',
'Field:Aggr 2 Reason Unable to Move',
'Field:Aggr 3 Threat Within Last 6 Months',
'Field:Women and Girls Severity Assessment',
'Field:DAPEV Info Rating',
'Field:Canadian Sponsorship Info Rating',
'Field:Aggr 1 Reason Unable to Move',
'Field:Nationality2',
'Field:Email Address1',
'Field:I 130 F',
'Field:Worked for USG',
'Field:Aggr 1 Mst Svr Only Threats',
'Medical_X_LPPN',
'Field:LPPN Score',
'Screening Decision',
'Date of Screening Start',
'Program Eligibilities',
'Date of Referral',
'First Referral',
'In IRAP Process']]

In [59]:
cleaned

,Vulnerability Threshold,Field:Aggr 3 Threats Since Moving,Field:RFR Submission,Field:US Prior Admission,Field:Aggr 3 Moved to Avoid Aggr,Field:Family Worked for USG,ILI Assistance Provided,Field:RFR,Field:Aggr 2 Moved to Avoid Aggr,Flags,...,process_day_>=7,process_days,Medical_X_LPPN,Field:LPPN Score,Screening Decision,Date of Screening Start,Program Eligibilities,Date of Referral,First Referral,In IRAP Process
0,Above,No,No,No,No,No,No,No,No,No,...,0.0,0.0,NaN,NaN,Cannot Represent,2019-08-12,Vulnerability-Based Relocation,2019-07-25,No,No
1,Below,No,No,No,No,No,No,No,No,No,...,NaN,NaN,NaN,NaN,NaN,NaT,Vulnerability-Based Relocation,2019-08-03,No,Yes
2,Below,No,No,No,No,No,No,No,No,No,...,NaN,NaN,NaN,NaN,NaN,2019-11-08,Vulnerability-Based RST and Protection,2019-08-08,Yes,No
3,Below,No,No,No,No,No,No,No,Yes,No,...,NaN,NaN,0.0,6.0,NaN,NaT,Vulnerability-Based Relocation,2019-08-02,No,No
4,Above,No,No,No,No,No,No,No,No,No,...,0.0,0.0,0.0,0.0,Cannot Represent,2019-12-20,Vulnerability-Based Relocation,2019-08-02,No,No
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7671,Below,No,No,No,No,No,No,No,No,No,...,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaT,NaN,NaN
7672,Below,No,No,No,No,No,No,No,No,No,...,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaT,NaN,NaN
7673,Below,No,No,No,No,No,No,No,No,No,...,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaT,NaN,NaN
7674,Below,No,No,No,No,No,No,No,No,No,...,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaT,NaN,NaN


In [60]:
null_count = pd.Series.to_frame(cleaned.isna().sum())
# null_count.rename(columns={"0": "null_count"})
null_count.columns = ["null_counts"]
null_count["total_counts"] = cleaned.shape[0]
null_count["null_percent"] = np.round(null_count["null_counts"] / cleaned.shape[0] , 5)
null_count["dtype"] = pd.Series.to_frame(cleaned.dtypes)
null_count["sample_value"] = cleaned.mode()[:1].T
null_count.sort_values(['null_counts'],ascending = False)[:20]

,null_counts,total_counts,null_percent,dtype,sample_value
Field:Handoff Timestamp,7170,7655,0.93664,object,2019-08-04 22:38:02.496664+03:00
process_day_>=1,6877,7655,0.89837,float64,0
process_days,6869,7655,0.89732,float64,0
process_day_>=7,6869,7655,0.89732,float64,0
Date of Screening Decision,6866,7655,0.89693,datetime64[ns],2019-07-26 00:00:00
Medical_X_LPPN,6846,7655,0.89432,float64,0
Field:LPPN Score,6840,7655,0.89353,float64,0
Screening Decision,6833,7655,0.89262,object,Cannot Represent
Client Location (Country),6828,7655,0.89197,object,Lebanon
Client Nationality,6811,7655,0.88975,object,Syrian


In [63]:
cleaned.to_csv("cleaned_data.csv")